## Notebook para la prueba de Porvenir Pensiones

##### Una de las principales ventajas de contar con datos estructurados en una serie temporal, es que nos permiten realizar analísis a través de herramientas estadísticas y econométricas como: evaluación de estacionariedad para las realizaciones de la muestra (¿la serie de tiempo -en adelante st, que describe el volumen de transacciones es determinística, estocástica, o un poco de ambos?), evaluación de correlación serial (¿la st para el volumen de transacciones tiene una tendencia predecible o cambia de dirección de manera abrupta entre cada periodo?), visualización de filtros para promedios móviles (¿podemos establecer estrategias respuesta a los movimientos de la st?), entre otras.

In [3]:
#Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import random
import statsmodels.api as sm

from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from sklearn.preprocessing import MinMaxScaler

from keras.layers import Input, Embedding, Dense, Flatten, Dropout, concatenate, LSTM
from keras.layers import BatchNormalization, SpatialDropout1D
from keras.callbacks import Callback
from keras.models import Model
from keras.optimizers import Adam

In [4]:
#Leer la base de datos
file = "data_01.txt"
data=pd.read_csv(file,sep='\t')

In [ ]:
#Revisar el tipo de datos que contiene el dataset
print(data.dtypes)
#Revisar sí hay missing values
print ('# de valores vacios:')
print (data.isnull().sum())

In [ ]:
#Convertir el tipo de datos de la variable fecha
data['fecha']=pd.to_datetime(data['fecha'])
print(data.dtypes)

In [ ]:
#Revisión a la cantidad de operaciones y de terminales
cantidadOperaciones=len(data.oper.unique())
print('Existen '+ str(cantidadOperaciones) + ' tipo de operaciones que maneja la entidad bancaria')
cantidadTerminales=len(data.idTerminal.unique())
print('Existen '+ str(cantidadTerminales) + ' terminales a cargo de la entidad bancaria')

In [ ]:
#Se elige las cinco operaciones con mayor frecuencia en los datos
tab = pd.crosstab(index=data["oper"],columns="frecuencia")
tab_orden=tab.sort_values('frecuencia',ascending=False)
print(tab_orden[0:5])

In [ ]:
#Se eligen cinco terminales al azar
auxTerm=data.idTerminal.unique()
terminalesAleat = random.choices(auxTerm,k=5)
print(terminalesAleat)
a=terminalesAleat[0]
b=terminalesAleat[1]
c=terminalesAleat[2]
d=terminalesAleat[3]
e=terminalesAleat[4]

In [ ]:
#Crear df 
terminal_a=dataf.query('idTerminal == @a and oper == 0',inplace=False)
print(terminal_a)
#Agrupar la frecuencia de las operaciones de manera diaria
terminal_as = terminal_a.groupby([terminal_a['fecha'].dt.date])['oper'].count().reset_index(name='Frecuencia')
print(terminal_as)
#Visualizar la st para el ensamblaje i 
terminal_as.plot(grid=True,figsize=(15,5))
plt.show()

In [ ]:
#Graficar contra la tendencia
terminal_as_ciclo, terminal_as_tend = sm.tsa.filters.hpfilter(terminal_as['Frecuencia'])
terminal_as['Tendencia'] = terminal_as_tend
terminal_as[['Frecuencia', 'Tendencia']].plot(figsize=(10, 8), fontsize=12)
legend = plt.legend()
legend.prop.set_size(14)
plt.show()

### Observaciones importantes
* Se debe generar periodos de tiempo comparables entre cada categoría de operación. Por lo cual, se opta por transformar la variable "fecha" de 'horas exactas' -lo cual emula una realización continua de la st,  a un periodo regular como lo puede ser 'diaria' -para obtener observaciones discretas de la st.
* Para trabajar el modelo de aprendizaje con variables categóricas, se requiere la incorporación de *embeddings* 

##### Considerando la naturaleza de la variable 'oper' se opta por 

In [ ]:
#Modelo de aprendizaje
terminal_as['weekday']=[x.weekday() for x in terminal_as.index]
terminal_as['month']=[x.month for x in terminal_as.index]
print(terminal_as)

PASOS=7
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    terminal_as = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(terminal_as.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(terminal_as.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
# load dataset
values = terminal_as['Frecuencia'].values

# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(-1, 1))

values=values.reshape(-1, 1) # esto lo hacemos porque tenemos 1 sola dimension

scaled = scaler.fit_transform(values)

reframed = series_to_supervised(scaled, PASOS, 1)
reframed.reset_index(inplace=True, drop=True)

contador=0
reframed['weekday']=terminal_as['weekday']
reframed['month']=terminal_as['month']

for i in range(reframed.index[0],reframed.index[-1]):
    reframed['weekday'].loc[contador]=terminal_as['weekday'][i+8]
    reframed['month'].loc[contador]=terminal_as['month'][i+8]
    contador=contador+1
print(reframed.head())

reordenado=reframed[ ['weekday','month','var1(t-7)','var1(t-6)','var1(t-5)','var1(t-4)','var1(t-3)','var1(t-2)','var1(t-1)','var1(t)'] ]
reordenado.dropna(inplace=True)

training_data = reordenado.drop('var1(t)',axis=1)#.values
target_data=reordenado['var1(t)']
#training_data.head()
print(reordenado)
numerito=len(reordenado)

valid_data = training_data[numerito-30:numerito]
valid_target=target_data[numerito-30:numerito]

training_data = training_data[0:numerito-30]
target_data=target_data[0:numerito-30]
print(training_data.shape,target_data.shape,valid_data.shape,valid_target.shape)
#training_data.head()

def crear_modeloEmbeddings():
    emb_dias = 2 #tamanio profundidad de embeddings
    emb_meses = 4

    in_dias = Input(shape=[1], name = 'dias')
    emb_dias = Embedding(7+1, emb_dias)(in_dias)
    in_meses = Input(shape=[1], name = 'meses')
    emb_meses = Embedding(12+1, emb_meses)(in_meses)

    in_cli = Input(shape=[PASOS], name = 'cli')

    fe = concatenate([(emb_dias), (emb_meses)])

    x = Flatten()(fe)
    x = Dense(PASOS,activation='tanh')(x)
    outp = Dense(1,activation='tanh')(x)
    model = Model(inputs=[in_dias,in_meses,in_cli], outputs=outp)

    model.compile(loss='mean_absolute_error', 
                  optimizer='adam',
                  metrics=['MSE'])

    model.summary()
    return model

EPOCHS=80

model = crear_modeloEmbeddings()

continuas=training_data[['var1(t-7)','var1(t-6)','var1(t-5)','var1(t-4)','var1(t-3)','var1(t-2)','var1(t-1)']]
valid_continuas=valid_data[['var1(t-7)','var1(t-6)','var1(t-5)','var1(t-4)','var1(t-3)','var1(t-2)','var1(t-1)']]

history=model.fit([training_data['weekday'],training_data['month'],continuas], target_data, epochs=EPOCHS
                 ,validation_data=([valid_data['weekday'],valid_data['month'],valid_continuas],valid_target))


results=model.predict([valid_data['weekday'],valid_data['month'],valid_continuas])
print( len(results) )
plt.scatter(range(len(valid_target)),valid_target,c='g')
plt.scatter(range(len(results)),results,c='r')
plt.title('validate')
plt.show()

In [ ]:
#Ventana con resultados
ventana=tk.Tk()
ventana.title('Inicio - Selección Operación')
ventana.geometry('500x400+400+300')
ventana.configure(background='white')

el=tk.Label(ventana,text='Nombre_',bg='green',fg='white')
el.pack(padx=5,pady=5,ipadx=5,ipady=5,fill=tk.X)
entrada1=tk.Entry(ventana)
var=tk.StringVar(ventana)
var.set('--Seleccione--')
opciones=tab_orden.oper[:5]
option=tk.OptionMenu(ventana,var,*opciones)
option.config(width=20)
option.pack(side='left',padx=30,pady=30)
e1=tk.Label(ventana,text='Animal seleccionado: ')
e1.pack(padx=5,pady=5,ipadx=5,ipady=5,fill=tk.X)
color=tk.Label(ventana,bg='plum',textvariable=var,pady=5,padx=5,width=50)
color.pack()

#entrada1.pack(fill=tk.X,padx=5,pady=5,ipadx=5,ipady=5)

image=tk.PhotoImage(file='descarga.gif')
image=image.subsample(1,1)
label=tk.Label(image=image)
label.pack()

ventana.mainloop()

#### *Referencias*

##### Montenegro, Alvaro *Análisis de Series de Tiempo*, Editorial Pontificia Universidad Javeriana